# Landsat Collection 2 on AWS

This notebook explores the Landsat Collection 2 data on AWS:

- [Landsat Collection 2 STAC API](https://landsatlook.usgs.gov/stac-server), a catalog of Landsat data
- [pystac-client](https://pystac-client.readthedocs.io/) for searching and access data
- [OpenDataCube](https://www.opendatacube.org/) and [odc-stac](https://odc-stac.readthedocs.io/) for loading STAC assets and representing geospatial data as XArrays
- [XArray](http://xarray.pydata.org/en/stable/), [pandas](https://pandas.pydata.org/) and [geopandas](https://geopandas.org/) for manipulating data
- [Dask](https://dask.org/) for performing parallel, distributed computing
- [Coiled.io](https://coiled.io/), a service for hosting Dask clusters
- [hvplot](https://hvplot.holoviz.org/) for visualization

Shown will be how to find data for an area of interest, explore the resulting metadata, perform calculations, and visualize the results.

Created by [Element 84](http://element84.com/)

In [5]:
!python3 -m pip install geopandas -I

Defaulting to user installation because normal site-packages is not writeable
  Using cached geopandas-1.0.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached numpy-2.2.5-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pyogrio-0.10.0-cp312-cp312-win_amd64.whl.metadata (5.6 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached pyproj-3.7.1-cp312-cp312-win_amd64.whl.metadata (31 kB)
  Using cached shapely-2.1.0-cp312-cp312-win_amd64.whl.metadata (7.0 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
Using cached geopandas-1.0.1-py3-none-any.whl (323 kB)
Using cached numpy-2.2.5-cp31

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.37.0 requires packaging<25,>=20, but you have packaging 25.0 which is incompatible.


In [6]:
# initial imports and reusable functions

# import holoviews as hv
# hv.extension('bokeh')

from copy import deepcopy
import geopandas as gpd
import hvplot.pandas
import pandas as pd
import pystac
from shapely.geometry import shape
import os
os.environ['AWS_REQUEST_PAYER'] = 'requester'

ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

ImportError: numpy._core.multiarray failed to import

In [ ]:
# create a function for later reuse
def plot_polygons(data, *args, **kwargs):
    return data.hvplot.paths(*args, geo=True, tiles='OSM', xaxis=None, yaxis=None,
                             frame_width=600, frame_height=600,
                             line_width=3, **kwargs)

# convert a list of STAC Items into a GeoDataFrame
def items_to_geodataframe(items):
    _items = []
    for i in items:
        _i = deepcopy(i)
        _i['geometry'] = shape(_i['geometry'])
        _items.append(_i)
    gdf = gpd.GeoDataFrame(pd.json_normalize(_items))
    for field in ['properties.datetime', 'properties.created', 'properties.updated']:
        if field in gdf:
            gdf[field] = pd.to_datetime(gdf[field])
    gdf.set_index('properties.datetime', inplace=True)
    return gdf

# set pystac_client logger to DEBUG to see API calls
import logging
logging.basicConfig()
logger = logging.getLogger('pystac_client')
logger.setLevel(logging.INFO)

# Search for data

Use pystac-client to find data in the Landsat STAC API. First, print up a table of all the STAC Collections in the API.

In [ ]:
# Open the Landsat STAC API

from pystac_client import Client
URL = 'https://landsatlook.usgs.gov/stac-server'
cat = Client.open(URL)
print(cat)

collections = [(c.id, c.title) for c in cat.get_collections()]
pd.set_option("display.max_colwidth", 150)
df = pd.DataFrame(collections, columns=['id', 'title'])
df

<Client id=stac-server>


,id,title
0,landsat-c2l2-sr,Landsat Collection 2 Level-2 UTM Surface Reflectance (SR) Product
1,landsat-c2l2-st,Landsat Collection 2 Level-2 UTM Surface Temperature (ST) Product
2,landsat-c2ard-st,Landsat Collection 2 Analysis Ready Data (ARD) Level-2 UTM Surface Temperature (ST) Product
3,landsat-c2l2alb-bt,Landsat Collection 2 Level-2 Albers Top of Atmosphere Brightness Temperature (BT) Product
4,landsat-c2l3-fsca,Landsat Collection 2 Level-3 Fractional Snow Covered Area (fSCA) Product
5,landsat-c2ard-bt,Landsat Collection 2 Analysis Ready Data (ARD) Level-2 UTM Top of Atmosphere Brightness Temperature (BT) Product
6,landsat-c2l1,Landsat Collection 2 Level-1 Product
7,landsat-c2l3-ba,Landsat Collection 2 Level-3 Burned Area (BA) Product
8,landsat-c2l2alb-st,Landsat Collection 2 Level-2 Albers Surface Temperature (ST) Product
9,landsat-c2ard-sr,Landsat Collection 2 Analysis Ready Data (ARD) Level-2 UTM Surface Reflectance (SR) Product


Fetch the collection of interest: Landsat Collection 2, Level 2 Surface Reflectance (landsat-c2l2-sr) and print the assets that are available.

In [ ]:
collection_id = 'landsat-c2l2-sr'

collection = cat.get_collection(collection_id)
pd.DataFrame.from_dict(collection.to_dict()['item_assets'], orient='index')

""


Change the AOI, search parameters here, and print how many matching scenes there are.

In [ ]:
import geopandas as gpd
import json

aoi = gpd.read_file('malawi.geojson')
geom = json.loads(aoi['geometry'].to_json())['features'][0]['geometry']

# limit sets the # of items per page so we can see multiple pages getting fetched
search = cat.search(
    collections = [collection_id],
    intersects = aoi['geometry'][0],
    datetime = "2021-01-01/2021-03-31",
    query = ["eo:cloud_cover<10"],
    limit = 100
)

print(f"{search.matched()} items found")

5 items found


# Use GeoPandas to view footprints

The cell below fetches all the STAC Items and updates the URLs to use the provided s3 URLs which can be used for direct access rather than the default https URLs. This is because the alternate extension is not yet supported in PySTAC, when it is there will be an easier way to specify which alternate URL, if any, to use for the assets.

Then, we create a GeoDataFrame for visualizing the footprints.

In [ ]:
# Get all items as a dictionary
items_dict = search.get_all_items_as_dict()['features']

# update URLs to use s3
for item in items_dict:
    for a in item['assets']:
        if 'alternate' in item['assets'][a] and 's3' in item['assets'][a]['alternate']:
            item['assets'][a]['href'] = item['assets'][a]['alternate']['s3']['href']
        item['assets'][a]['href'] = item['assets'][a]['href'].replace('usgs-landsat-ard', 'usgs-landsat')

# Create GeoDataFrame from Items
items_gdf = items_to_geodataframe(items_dict)

print(f"{len(items_dict)} items found")

pd.reset_option("display.max_colwidth")
items_gdf.head()

c:\Users\Khizer Zakir\.conda\envs\virtucube\Lib\site-packages\pystac_client\item_search.py:911: FutureWarning: get_all_items_as_dict() is deprecated, use item_collection_as_dict() instead.
  warnings.warn(


5 items found


,type,stac_version,stac_extensions,id,description,bbox,geometry,links,collection,properties.eo:cloud_cover,...,assets.qa_aerosol.title,assets.qa_aerosol.description,assets.qa_aerosol.type,assets.qa_aerosol.roles,assets.qa_aerosol.classification:bitfields,assets.qa_aerosol.href,assets.qa_aerosol.alternate.s3.storage:platform,assets.qa_aerosol.alternate.s3.storage:requester_pays,assets.qa_aerosol.alternate.s3.href,assets.qa_aerosol.file:checksum
properties.datetime,,,,,,,,,,,,,,,,,,,,,
2021-03-27 06:53:04.348512+00:00,Feature,1.0.0,[https://landsat.usgs.gov/stac/landsat-extensi...,LE07_L2SP_167071_20210327_20210422_02_T1_SR,Landsat Collection 2 Level-2 Surface Reflectan...,"[34.44991343201458, -16.83821593812492, 36.617...","POLYGON ((34.81393 -14.96579, 34.44991 -16.576...","[{'rel': 'self', 'href': 'https://landsatlook....",landsat-c2l2-sr,5.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-26 07:46:58.363578+00:00,Feature,1.0.0,[https://landsat.usgs.gov/stac/landsat-extensi...,LC08_L2SP_168071_20210326_20210402_02_T1_SR,Landsat Collection 2 Level-2 Surface Reflectan...,"[32.92416142339678, -16.94629631879435, 35.016...","POLYGON ((33.31912 -14.85778, 32.92416 -16.586...","[{'rel': 'self', 'href': 'https://landsatlook....",landsat-c2l2-sr,1.37,...,Aerosol Quality Analysis Band,Collection 2 Level-2 Aerosol Quality Analysis ...,image/vnd.stac.geotiff; cloud-optimized=true,"[metadata, data-mask, water-mask]","[{'name': 'fill', 'description': 'Correspondin...",s3://usgs-landsat/collection02/level-2/standar...,AWS,True,s3://usgs-landsat/collection02/level-2/standar...,1340d449c994635d4fac33e619d168f731898600360ea1...
2021-01-28 07:53:05.529852+00:00,Feature,1.0.0,[https://landsat.usgs.gov/stac/landsat-extensi...,LC08_L2SP_169070_20210128_20210305_02_T1_SR,Landsat Collection 2 Level-2 Surface Reflectan...,"[31.711090518856604, -15.502202043278283, 33.7...","POLYGON ((33.40973 -15.5022, 33.78616 -13.7680...","[{'rel': 'self', 'href': 'https://landsatlook....",landsat-c2l2-sr,6.59,...,Aerosol Quality Analysis Band,Collection 2 Level-2 Aerosol Quality Analysis ...,image/vnd.stac.geotiff; cloud-optimized=true,"[metadata, data-mask, water-mask]","[{'name': 'fill', 'description': 'Correspondin...",s3://usgs-landsat/collection02/level-2/standar...,AWS,True,s3://usgs-landsat/collection02/level-2/standar...,13407ac8f384d8dcdf379ec7e812b7548ca6775a4c93df...
2021-01-06 06:59:30.874396+00:00,Feature,1.0.0,[https://landsat.usgs.gov/stac/landsat-extensi...,LE07_L2SP_167072_20210106_20210201_02_T1_SR,Landsat Collection 2 Level-2 Surface Reflectan...,"[34.128584545965374, -18.284720058135903, 36.3...","POLYGON ((34.4956 -16.41165, 34.12858 -18.0166...","[{'rel': 'self', 'href': 'https://landsatlook....",landsat-c2l2-sr,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-06 06:59:06.938212+00:00,Feature,1.0.0,[https://landsat.usgs.gov/stac/landsat-extensi...,LE07_L2SP_167071_20210106_20210201_02_T1_SR,Landsat Collection 2 Level-2 Surface Reflectan...,"[34.45806605660863, -16.838379235727494, 36.62...","POLYGON ((34.82285 -14.96626, 34.45807 -16.576...","[{'rel': 'self', 'href': 'https://landsatlook....",landsat-c2l2-sr,8.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
plot_polygons(aoi) * items_gdf.hvplot.paths(geo=True)

:Overlay
   .WMTS.I  :WMTS   [Longitude,Latitude]
   .Path.I  :Path   [Longitude,Latitude]
   .Path.II :Path   [Longitude,Latitude]

# OpenDataCube

Now we'll turn the set of scenes into a virtual datacube. None of the data will actually be read yet.

The configuration string (`cfg`) is for providing additional info not currently available in the STAC Items, but will be in the future.

In [ ]:
import yaml

cfg = """---
landat-c2l2-sr:
  measurements:
    '*':
      dtype: uint16
      nodata: 0
      unit: 'm'
"""
cfg = yaml.load(cfg, Loader=yaml.CSafeLoader)

Here we load as a DataCube. A PySTAC ItemCollection is created from the found STAC Items, and we specify various parameters, such as bands of interest, chunk size, and geometry to clip the data to.

In [7]:
%%time

from odc.stac import stac_load

# Create PySTAC ItemCollection
item_collection = pystac.ItemCollection(items_dict)

# default to CRS and resolution from first Item
from pystac.extensions.projection import ProjectionExtension
from pyproj import CRS

dc = stac_load(item_collection,
               bands=['red', 'blue', 'green', 'nir08'],
               chunks={"x": 2048, "y": 2048},
               groupby='solar_day',
               stac_cfg=cfg,
               geopolygon=geom,
)
dc

ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

ImportError: numpy._core.multiarray failed to import

# Calculations

We create an RGB datacube representation and generate an NDVI datacube.

In [ ]:
vis = dc.odc.to_rgba(vmin=1, vmax=20000, bands=['red', 'green', 'blue'])
vis

In [ ]:
ndvi = ((dc['nir08'] - dc['red']) / (dc['nir08'] + dc['red'])).clip(0, 1)
ndvi.name = 'ndvi'
ndvi

# Start Dask Client

Start either a local Dask, or use [coiled.io](coiled.io)

In [ ]:
# local Dask

from dask.distributed import Client
client = Client()
client

# Compute

Now, we kick off our Dask computation by using the Dask persist function, which will keep the data in memory on the cluster for faster access later.

The Dask `compute` function is used when we actually want the data, such as displaying it.

In [ ]:
client

In [ ]:
%%time
from dask.distributed import wait

vis = client.persist(vis)
_ = wait(vis)

In [ ]:
%%time
vis_ = vis.compute()
vis_.plot.imshow(col='time', rgb='band', col_wrap=5, robust=True)

In [ ]:
import hvplot.xarray
import panel as pn

In [ ]:
vis_.spatial_ref.crs_wkt

In [ ]:
hvplot_kwargs = dict(crs=32610, tiles='OSM', 
                frame_width=400, widgets={'time': pn.widgets.Select})

vis_.hvplot.rgb(x='x', y='y', bands='band', **hvplot_kwargs)

In [ ]:
vis_.hvplot.rgb(x='x', y='y', bands='band', **hvplot_kwargs)

In [ ]:
%%time
ndvi_ = ndvi.compute()
ndvi_.hvplot(x='x', y='y', **hvplot_kwargs)

Create an animated GIF of NDVI over time using `geogif` with the fetched results.

In [ ]:
from geogif import gif

#ndvi_ = ndvi_c.transpose('time','x','y').compute()

gif(ndvi_, fps=1, cmap='YlGn')

In [ ]:
%%time
ndvi_mean = ndvi.mean(dim=['x', 'y']).compute()
ndvi_mean.hvplot()

# Shutdown cluster

Shut down the cluster.

In [ ]:
client.close()